In [ ]:
import numpy as np

In [ ]:
print("hello World!")
print("HELL YEA")

In [ ]:
x = 0.1
y = 0.3



In [ ]:
if x < y:
    print("x t'as fumé ta race!!!!")

# J'ai juste créer ce petit notebook pour voir si on allait réussir a utiliser Jupyter notebook et Git de manière stylée cette fois

In [ ]:
import numpy as np
import pandas as pd
import re # Used for the conversion of "r##_c##" in only the numbers --> TODO: check where it comes from
from helpers import *
from play_with_data import *


In [ ]:
#********* Load the given data **********
r_c, x = load_data('./data_train.csv')

In [ ]:
#********* Creation of a matrix of the data ********
nUser = 10000
nItem = 1000
data = np.zeros([nUser, nItem]) # These numbers were given

for ind, i in enumerate(r_c): # Loop over all the ID 
    data[int(re.findall('\d+', i)[0])-1, int(re.findall('\d+', i)[1])-1] = x[ind] # Use the information in the ID (row, col) to create the matrix




In [ ]:
df = pd.DataFrame(data)
a = []
for i in range(df.shape[0]):
    a.append(len(np.where(df.loc[i] != 0)[0]))
#print(a)
#print(len(a))
print(np.std(a))
print(np.mean(a))
#display(df)

In [ ]:
# Check if there is any missed data 
# It was told us that we have the data from 10'000 users for 1000 films, but we don't have all these data
info_general(nUser, nItem, x, data)
print('\n')
info_ratings(data)


In [ ]:
#********** Data preview ************
